<a href="https://colab.research.google.com/github/eric-pding/RSM8415-Service-Analytics/blob/main/RSM8415_Case3_Eric_ver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RSM8415-Case3
Eric Ding



# Import all stuff

In [ ]:
# Install
!pip3 install googletrans==3.1.0a0
!pip install vaderSentiment
!pip install autocorrect
!pip install deep_translator
!pip install researchpy
!pip install textblob
!pip install afinn

In [ ]:
!pip install scikit-learn==1.2.2
!pip install imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

In [ ]:
# Class 1 Code
import warnings
warnings.filterwarnings('ignore')

import os
import re
import pandas as pd
import numpy as np
import seaborn as sns
import researchpy as rp
import nltk

from autocorrect import Speller
from googletrans import Translator, constants
from matplotlib import pyplot as plt
from deep_translator import GoogleTranslator

# NLTK - download
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
#from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix,roc_curve,auc

from itertools import cycle
from textblob import TextBlob

# Class 2 Code
#!pip3 install googletrans==3.1.0a0

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Read data in

In [ ]:
github_path = "https://github.com/eric-pding/RSM8415-Service-Analytics/raw/main/Case3/case3.xlsx"
df = pd.read_excel(github_path, sheet_name = 'data')


,REVIEW ID,SOURCE,REVIEW BY,REVIEW DATE,REVIEW SUBJECT,text,REVIEW RATING,REVIEW TYPE
0,814,Trip Advisor,maryd1928,2013-01-02,Don't miss this Bangalore Temp,Don't miss this Gaudiya Vaishnava Temple locat...,5,POSITIVE
1,1181,Trip Advisor,maryd1928,2013-01-02,Don't miss this Bangalore Temple,Don't miss this Gaudiya Vaishnava Temple locat...,5,POSITIVE
2,813,Trip Advisor,shoubhik,2013-01-05,peacefull and devotional,This was my 2nd visit to the temple.. last tim...,4,POSITIVE
3,1182,Trip Advisor,shoubhik,2013-01-05,peacefull and devotional,This was my 2nd visit to the temple.. last tim...,4,POSITIVE
4,812,Trip Advisor,onthemoveBangalore,2013-01-10,Huge and Grand,The Iskcon temple is a rather grand and imposi...,5,POSITIVE
...,...,...,...,...,...,...,...,...
5680,5685,Trip Advisor,disheet p,2017-08-11,Feeling blessed,Good,5,POSITIVE
5681,5682,Google + HK HILL,Gangaraju L,2017-08-11,NaN,Excellent,4,POSITIVE
5682,5684,Google + HK HILL,R Sivakumar,2017-08-11,NaN,Visited with friends on 12 Aug 2017. Not much ...,5,POSITIVE
5683,5686,Trip Advisor,Lalengmawia V,2017-08-12,Not interesting,Its a very beautiful temple . if you are a tou...,3,NEUTRAL


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5685 entries, 0 to 5684
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   REVIEW ID       5685 non-null   int64         
 1   SOURCE          5685 non-null   object        
 2   REVIEW BY       5684 non-null   object        
 3   REVIEW DATE     5685 non-null   datetime64[ns]
 4   REVIEW SUBJECT  3773 non-null   object        
 5   text            5678 non-null   object        
 6   REVIEW RATING   5685 non-null   int64         
 7   REVIEW TYPE     5685 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 355.4+ KB


In [ ]:
df.describe()

,REVIEW ID,REVIEW DATE,REVIEW RATING
count,5685.000000,5685,5685.000000
mean,2844.292700,2016-01-04 00:27:21.372031744,4.404749
min,2.000000,2013-01-02 00:00:00,0.000000
25%,1423.000000,2015-02-05 00:00:00,4.000000
50%,2844.000000,2016-03-21 00:00:00,5.000000
75%,4266.000000,2017-02-13 00:00:00,5.000000
max,5687.000000,2017-08-13 00:00:00,5.000000
std,1641.621108,NaN,0.865281


In [ ]:
df.head(10)

,REVIEW ID,SOURCE,REVIEW BY,REVIEW DATE,REVIEW SUBJECT,text,REVIEW RATING,REVIEW TYPE
0,814,Trip Advisor,maryd1928,2013-01-02,Don't miss this Bangalore Temp,Don't miss this Gaudiya Vaishnava Temple locat...,5,POSITIVE
1,1181,Trip Advisor,maryd1928,2013-01-02,Don't miss this Bangalore Temple,Don't miss this Gaudiya Vaishnava Temple locat...,5,POSITIVE
2,813,Trip Advisor,shoubhik,2013-01-05,peacefull and devotional,This was my 2nd visit to the temple.. last tim...,4,POSITIVE
3,1182,Trip Advisor,shoubhik,2013-01-05,peacefull and devotional,This was my 2nd visit to the temple.. last tim...,4,POSITIVE
4,812,Trip Advisor,onthemoveBangalore,2013-01-10,Huge and Grand,The Iskcon temple is a rather grand and imposi...,5,POSITIVE
5,1183,Trip Advisor,onthemoveBangalore,2013-01-10,Huge and Grand,The Iskcon temple is a rather grand and imposi...,5,POSITIVE
6,808,Trip Advisor,RobertBorchers,2013-01-12,Interesting temple for foreign,Interesting temple with lots of shop. Very dif...,4,POSITIVE
7,1184,Trip Advisor,RobertBorchers,2013-01-12,Interesting temple for foreigners,Interesting temple with lots of shop. Very dif...,4,POSITIVE
8,762,Trip Advisor,shariffmim,2013-01-14,peace..,had been to dis place with my frnds and got to...,5,POSITIVE
9,1185,Trip Advisor,shariffmim,2013-01-14,peace..,had been to dis place with my frnds and got to...,5,POSITIVE


# Q1
Identify and Keep the First Occurrence of Duplicate Reviews by the Same Person (identified by REVIEW BY) on the Same Day.

In [ ]:
df_q1 = df.sort_values(by=['REVIEW BY','REVIEW DATE'], ascending=True)
df_q1 = df_q1.drop_duplicates(subset=['REVIEW BY', 'REVIEW DATE'], keep='first')
df_q1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4645 entries, 629 to 3426
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   REVIEW ID       4645 non-null   int64         
 1   SOURCE          4645 non-null   object        
 2   REVIEW BY       4644 non-null   object        
 3   REVIEW DATE     4645 non-null   datetime64[ns]
 4   REVIEW SUBJECT  2769 non-null   object        
 5   text            4641 non-null   object        
 6   REVIEW RATING   4645 non-null   int64         
 7   REVIEW TYPE     4645 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 326.6+ KB


# Q2
Identify and Keep the First Occurrence of Reviews by the Same Person (identified by REVIEW BY) Within a 7-day Interval.

In [ ]:
# Define a function
def filter_first_within_7_days(group):
    filtered_reviews = []
    prev_date = None
    for index, row in group.iterrows():
        if prev_date is None or (row['REVIEW DATE'] - prev_date).days > 7:
            filtered_reviews.append(row)
            prev_date = row['REVIEW DATE']
    return pd.DataFrame(filtered_reviews)

# Group by 'REVIEW BY' and apply the filtering function
filtered_groups = df_q1.groupby('REVIEW BY', group_keys=False).apply(filter_first_within_7_days)
# Reset index
df_q2 = filtered_groups.reset_index(drop=True)
# show info()
df_q2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4632 entries, 0 to 4631
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   REVIEW ID       4632 non-null   int64         
 1   SOURCE          4632 non-null   object        
 2   REVIEW BY       4632 non-null   object        
 3   REVIEW DATE     4632 non-null   datetime64[ns]
 4   REVIEW SUBJECT  2760 non-null   object        
 5   text            4628 non-null   object        
 6   REVIEW RATING   4632 non-null   int64         
 7   REVIEW TYPE     4632 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 289.6+ KB
